In [1]:
# データ分析・操作用ライブラリ
import numpy as np
import pandas as pd

# NLP用ライブラリ
import MeCab,re
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import word2vec

# スコア評価用ライブラリ
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support

# エラー表示の抑制
import warnings

In [2]:
# 表示関連
# DataFrameの列数設定
pd.set_option('display.max_columns', 500)
pd.set_option("display.max_rows", 500)
warnings.filterwarnings('ignore')

In [3]:
# 形態素解析用関数
def tokenize(text):
    tokens = []
    tagger = MeCab.Tagger()
    node = tagger.parseToNode(text)
    while node:
        if node.feature.split(",")[0] in ['名詞'] :
                replace_node = re.sub( re.compile( "[!-/:-@[-`{-~]" ), "", node.surface )
                if replace_node != "" and replace_node != " ":
                    tokens.append(replace_node)
        node = node.next
    return tokens

In [4]:
def MeanEmbeddingVectorizer(text, model):
    #vectors = [keyed_vec.get_vector(token) for token in tokens if token in model.vocab]
    mean_vectors = []
    tagger = MeCab.Tagger()
    node = tagger.parseToNode(text)
    while node:
        if node.feature.split(",")[0] in ['名詞'] :
                replace_node = re.sub( re.compile( "[!-/:-@[-`{-~]" ), "", node.surface )
                if replace_node != "" and replace_node != " ":
                    if replace_node in model.vocab:
                        mean_vectors.append(np.mean(model.get_vector(replace_node)))
                    else:
                        mean_vectors.append(0.0)
        node = node.next
    return np.mean(mean_vectors)

In [5]:
# テキストデータの読み込み
wikiData = pd.read_csv("csv/train_cleaned.csv", encoding="utf-8")

# 読み込みデータの表示
wikiData.head()

,text,target
0,ドール・フード・カンパニー ドール・フード・カンパニー Dole Food Comp...,1
1,南ヶ丘牧場 株式会社 南ヶ丘牧場 みなみ じょう 栃木県 那須高原 本拠 酪...,1
2,岩瀬牧場 岩瀬牧場 ぼくじょう 日本 福島県 岩瀬 郡 鏡石町 牧場 鏡石...,1
3,くら ぎ GI Co Ltd 三重県 中心 農業 店舗 展開 会社 本...,1
4,ハッピーネモファーム 株式会社 ハッピーネモファーム 北海道 浦河 郡 浦河町 ...,1


In [6]:
# Numpy配列へ変換
X = wikiData.text.values
y = wikiData.target.values

In [7]:
# トレーニングデータ、テストデータの分割
X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.2, random_state=42, stratify=y)

In [8]:
# TF-IDFへ変換
vectorizer = TfidfVectorizer(tokenizer=tokenize)
vectorizer.fit(X)
train_array = vectorizer.transform(X_train).toarray()
test_array = vectorizer.transform(X_test).toarray()

In [9]:
import gensim.models.keyedvectors as word2vec_for_txt
model_dir = 'model/fastText.model.vec'
model = word2vec_for_txt.KeyedVectors.load_word2vec_format(model_dir, binary=False)

In [16]:
for i in range(len(X_train)):
    X_train[i] = MeanEmbeddingVectorizer(X_train[i], model)
    
for i in range(len(X_test)):
    X_test[i] = MeanEmbeddingVectorizer(X_test[i], model)

In [17]:
work_pd1 = pd.concat([pd.DataFrame(train_array),pd.DataFrame(X_train)],axis=1)
work_pd2 = pd.concat([pd.DataFrame(train_array),pd.DataFrame(X_train)],axis=1)

In [ ]:
X_train2 = work_pd1.values
X_text2 = work_pd2.values

In [ ]:
# Light GBM
import lightgbm as lgb

clf = lgb.LGBMClassifier(objective='multiclass', num_class=33)
clf.fit(train_array, y_train)
cv_scores = cross_val_score(clf, train_array, y_train, cv=5)

print("Training score：" + str(clf.score(train_array, y_train)))
print("Cross-Validation score：" + str(np.mean(cv_scores)))
print("Test score：" + str(clf.score(test_array, y_test)))